In [47]:
!pip install streamlit pyngrok xlsxwriter

In [51]:
import pandas as pd
from google.colab import files

def assign_week(date):

    day = date.day
    if 1 <= day <= 7:
        return "01 Feb - 07 Feb"
    elif 8 <= day <= 14:
        return "08 Feb - 14 Feb"
    elif 15 <= day <= 21:
        return "15 Feb - 21 Feb"
    elif 22 <= day <= 28:
        return "22 Feb - 28 Feb"
    else:
        return "29 Feb - ..."  # Adjust if needed

def compute_weekly_chat_metrics(df):

    # Overall metrics for the month:
    overall_incoming = df['RoomCode'].count()
    overall_unique = df['UserId'].nunique()
    overall_closed_bot = df[df['ClosedBy'] == 'System'].shape[0]
    overall_deflection = round((overall_closed_bot / overall_incoming * 100), 2) if overall_incoming > 0 else 0
    overall_closed_agent = df[df['ClosedBy'] != 'System'].shape[0]

    overall_row = {
        "Week": "Overall",
        "Incoming Chats": overall_incoming,
        "Unique Users": overall_unique,
        "Closed By Bot": overall_closed_bot,
        "Bot Deflection %": overall_deflection,
        "Closed By Agents": overall_closed_agent
    }

    # Create a new column 'Week' based on ChatStartTime.
    df['Week'] = df['ChatStartTime'].apply(assign_week)

    # Group by week and calculate metrics.
    weekly_rows = []
    for week, group in df.groupby("Week"):
        incoming = group['RoomCode'].count()
        unique = group['UserId'].nunique()
        closed_bot = group[group['ClosedBy'] == 'System'].shape[0]
        deflection = round((closed_bot / incoming * 100), 2) if incoming > 0 else 0
        closed_agent = group[group['ClosedBy'] != 'System'].shape[0]
        weekly_rows.append({
            "Week": week,
            "Incoming Chats": incoming,
            "Unique Users": unique,
            "Closed By Bot": closed_bot,
            "Bot Deflection %": deflection,
            "Closed By Agents": closed_agent
        })
    weekly_df = pd.DataFrame(weekly_rows)

    # Order the weekly rows by desired week order.
    desired_order = ["01 Feb - 07 Feb", "08 Feb - 14 Feb", "15 Feb - 21 Feb", "22 Feb - 28 Feb"]
    # Convert "Week" to a categorical type with our order for proper sorting:
    weekly_df['Week'] = pd.Categorical(weekly_df['Week'], categories=desired_order, ordered=True)
    weekly_df = weekly_df.sort_values("Week")

    # Concatenate the overall row and weekly rows.
    overall_df = pd.DataFrame([overall_row])
    final_df = pd.concat([overall_df, weekly_df], ignore_index=True)
    return final_df

def main():
    # File in Colab (ensure your file is uploaded to /content folder with this name)
    file_path = "/content/DataDump.csv"

    try:
        df = pd.read_csv(file_path)
        print("Data loaded successfully!")
    except Exception as e:
        print("Error reading DataDump.csv:", e)
        return

    # Convert ChatStartTime and ChatEndTime using dayfirst=True
    df['ChatStartTime'] = pd.to_datetime(df['ChatStartTime'], dayfirst=True)
    df['ChatEndTime'] = pd.to_datetime(df['ChatEndTime'], dayfirst=True)

    # Optional: Convert AgentFirstResponseTime if needed (using errors='coerce')
    df['AgentFirstResponseTime'] = pd.to_timedelta(df['AgentFirstResponseTime'], errors='coerce')

    # Compute the weekly chat metrics
    metrics_df = compute_weekly_chat_metrics(df)
    print(metrics_df)

    # Write results to an Excel file
    output_file = "weekly_chat_metrics.xlsx"
    metrics_df.to_excel(output_file, index=False)
    print("Weekly chat metrics report generated as:", output_file)

    # Download the Excel file
    files.download(output_file)

# Run the main function in Colab.
main()


Data loaded successfully!
              Week  Incoming Chats  Unique Users  Closed By Bot  \
0          Overall           10000          9404           6268   
1  01 Feb - 07 Feb            2763          2642           1788   
2  08 Feb - 14 Feb            2620          2513           1668   
3  15 Feb - 21 Feb            2286          2188           1364   
4  22 Feb - 28 Feb            2329          2230           1446   
5              NaN               2             2              2   

   Bot Deflection %  Closed By Agents  
0             62.68              3732  
1             64.71               975  
2             63.66               952  
3             59.67               922  
4             62.09               883  
5            100.00                 0  
Weekly chat metrics report generated as: weekly_chat_metrics.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
from pyngrok import ngrok
import subprocess
import time

# Kill any existing ngrok tunnels if needed (optional)
!pkill ngrok

# Add your ngrok authtoken to the configuration
!ngrok config add-authtoken 2wFfJmoGHjhTrw158QdVssKwiRk_2Dxs7mtwShJFGsJVhBrLW  # Replaced <your_authtoken> with your actual token

# Get a public URL for port 8501 (default Streamlit port)
# Remove or comment out config_path and use the 'port' argument directly
tunnel = ngrok.connect(8501, proto="http")
public_url = tunnel.public_url
print("Public URL:", public_url)

# Run Streamlit app in the background
proc = subprocess.Popen(["streamlit", "run", "app.py"])

# Wait for Streamlit to start
time.sleep(5)
print("The Streamlit app should now be accessible via the public URL above.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://8156-34-57-32-240.ngrok-free.app
The Streamlit app should now be accessible via the public URL above.
